In [8]:
import numpy as np
import pandas as pd

In [2]:
!pip install groq


     -------------------------------------- 138.3/138.3 kB 1.4 MB/s eta 0:00:00
     -------------------------------------- 73.5/73.5 kB 506.2 kB/s eta 0:00:00
     -------------------------------------- 71.0/71.0 kB 653.4 kB/s eta 0:00:00
     ---------------------------------------- 78.8/78.8 kB 1.5 MB/s eta 0:00:00
  Attempting uninstall: idna
    Found existing installation: idna 2.6
    Uninstalling idna-2.6:
      Successfully uninstalled idna-2.6
  Attempting uninstall: h11
    Found existing installation: h11 0.9.0
    Uninstalling h11-0.9.0:
      Successfully uninstalled h11-0.9.0
  Attempting uninstall: httpcore
    Found existing installation: httpcore 0.9.1
    Uninstalling httpcore-0.9.1:
      Successfully uninstalled httpcore-0.9.1
  Attempting uninstall: httpx
    Found existing installation: httpx 0.13.3
    Uninstalling httpx-0.13.3:
      Successfully uninstalled httpx-0.13.3


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
googletrans 4.0.0rc1 requires httpx==0.13.3, but you have httpx 0.28.1 which is incompatible.
conda-repo-cli 1.0.20 requires clyent==1.2.1, but you have clyent 1.2.2 which is incompatible.
conda-repo-cli 1.0.20 requires nbformat==5.4.0, but you have nbformat 5.10.2 which is incompatible.
conda-repo-cli 1.0.20 requires PyYAML==6.0, but you have pyyaml 6.0.2 which is incompatible.
conda-repo-cli 1.0.20 requires requests==2.28.1, but you have requests 2.32.3 which is incompatible.


In [9]:


import pandas as pd
from groq import Groq
from tqdm import tqdm


In [10]:
df=pd.read_csv(r"C:\Users\MAITHILI\Gen-AI_FraudGuard\Datasets\paysim_processed.csv")
df.head()

,transaction_id,step,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,...,type_DEBIT,type_PAYMENT,type_TRANSFER,ml_anomaly_flag,ml_fraud_score,rule_score,rule_reasons,rule_score_norm,total_fraud_score,flagged_fraud
0,0,278,330218.42,C632336343,20866.00,351084.42,C834976624,452419.57,122201.15,0,...,False,False,False,0,0.256915,30,Large transaction amount,0.3,0.274149,0
1,1,15,11647.08,C1264712553,30370.00,18722.92,M215391829,0.00,0.00,0,...,False,True,False,0,0.003595,0,NaN,0.0,0.002157,0
2,2,10,152264.21,C1746846248,106589.00,258853.21,C1607284477,201303.01,49038.80,0,...,False,False,False,0,0.107625,0,NaN,0.0,0.064575,0
3,3,403,1551760.63,C333676753,0.00,0.00,C1564353608,3198359.45,4750120.08,0,...,False,False,True,0,0.495751,60,High-risk transaction type; Large transaction ...,0.6,0.537450,0
4,4,206,78172.30,C813403091,2921331.58,2999503.88,C1091768874,415821.90,337649.60,0,...,False,False,False,0,0.224300,0,NaN,0.0,0.134580,0


In [11]:
#uncomment below line and enter your own api_key
#client = Groq(api_key="*******")


In [12]:
df[["amount", "ml_fraud_score", "rule_score", "total_fraud_score", "flagged_fraud"]].head()

,amount,ml_fraud_score,rule_score,total_fraud_score,flagged_fraud
0,330218.42,0.256915,30,0.274149,0
1,11647.08,0.003595,0,0.002157,0
2,152264.21,0.107625,0,0.064575,0
3,1551760.63,0.495751,60,0.537450,0
4,78172.30,0.224300,0,0.134580,0


In [13]:
df['flagged_fraud'].value_counts()

flagged_fraud
0    149619
1       381
Name: count, dtype: int64

In [14]:
def investigate_transaction(row):
    
    balance_org_change = row.newbalanceOrig - row.oldbalanceOrg
    balance_dest_change = row.newbalanceDest - row.oldbalanceDest

    prompt = f"""
You are a senior financial fraud investigator.
Write for a fraud analyst who has limited time and needs clarity.

Transaction details:
- Amount: {row.amount}
- Transaction Type: {"TRANSFER" if row.type_TRANSFER else "CASH_OUT" if row.type_CASH_OUT else "OTHER"}

Account balances:
- Sender balance before: {row.oldbalanceOrg}
- Sender balance after: {row.newbalanceOrig}
- Receiver balance before: {row.oldbalanceDest}
- Receiver balance after: {row.newbalanceDest}

Balance changes:
- Sender balance change: {balance_org_change} (negative means loss)
- Receiver balance change: {balance_dest_change} (positive means gain)

Model signals:
- ML Fraud Score (0–1): {row.ml_fraud_score}
- Rule Score (0–100): {row.rule_score}
- Total Fraud Score (0–1): {row.total_fraud_score}

Instructions:
1. Assign a Risk Level using this rule:
   - Low: < 0.5
   - Medium: 0.5–0.7
   - High: > 0.7

2. Respond in the EXACT format below. Do not add extra text.

Risk Level: <Low | Medium | High>

Key Risk:
<1–2 concise sentences explaining the strongest reasons this transaction is risky or not risky.>

Recommended Action:
<1–2 concise sentences describing what the fraud team should do next.>

Rules:
- Be factual and professional.
- Do NOT use bullet points.
- Do NOT speculate beyond the provided data.
"""

    response = client.chat.completions.create(
        model="llama-3.1-8b-instant",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.2
    )

    return response.choices[0].message.content



In [15]:

flagged_df = df[df["flagged_fraud"] == 1].copy()


llm_explanations = []

for idx, row in tqdm(flagged_df.iterrows(), total=flagged_df.shape[0]):
    explanation = investigate_transaction(row)
    llm_explanations.append(explanation)

flagged_df["llm_explanation"] = llm_explanations


100%|██████████| 381/381 [26:32<00:00,  4.18s/it]


In [16]:
flagged_df['llm_explanation'].head()

331     Risk Level: High\n\nKey Risk: The transaction ...
1091    Risk Level: High\n\nKey Risk: The high ML Frau...
1345    Risk Level: High\n\nKey Risk: The transaction ...
1632    Risk Level: High\n\nKey Risk: The transaction ...
2530    Risk Level: High\n\nKey Risk: The transaction'...
Name: llm_explanation, dtype: object

In [17]:
df["llm_explanation"] = "Not investigated (low risk)"


In [18]:
df.loc[flagged_df.index, "llm_explanation"] = flagged_df["llm_explanation"]


In [19]:
for i, text in enumerate(flagged_df["llm_explanation"].head(5)):
    print(f"\n--- LLM Explanation #{i+1} ---")
    print(text)



--- LLM Explanation #1 ---
Risk Level: High

Key Risk: The transaction amount is significantly large, resulting in the sender's account balance dropping to zero, and the receiver's account balance increasing by over 4.1 million. The high ML Fraud Score and Rule Score also indicate a high risk of fraud.

Recommended Action: Review the transaction details and account activity to identify potential suspicious patterns and investigate the legitimacy of the transaction.

--- LLM Explanation #2 ---
Risk Level: High

Key Risk: The high ML Fraud Score of 0.778405089095148 and Rule Score of 100 indicate a strong likelihood of fraudulent activity. The significant transfer amount of 5279372.81 from a sender with a balance of 2527.46 to a receiver with a balance of 6197777.7 also raises suspicions.

Recommended Action: The fraud team should investigate the sender's and receiver's accounts for any suspicious activity and verify the legitimacy of the transaction.

--- LLM Explanation #3 ---
Risk Le

In [20]:
df.head()

,transaction_id,step,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,...,type_PAYMENT,type_TRANSFER,ml_anomaly_flag,ml_fraud_score,rule_score,rule_reasons,rule_score_norm,total_fraud_score,flagged_fraud,llm_explanation
0,0,278,330218.42,C632336343,20866.00,351084.42,C834976624,452419.57,122201.15,0,...,False,False,0,0.256915,30,Large transaction amount,0.3,0.274149,0,Not investigated (low risk)
1,1,15,11647.08,C1264712553,30370.00,18722.92,M215391829,0.00,0.00,0,...,True,False,0,0.003595,0,NaN,0.0,0.002157,0,Not investigated (low risk)
2,2,10,152264.21,C1746846248,106589.00,258853.21,C1607284477,201303.01,49038.80,0,...,False,False,0,0.107625,0,NaN,0.0,0.064575,0,Not investigated (low risk)
3,3,403,1551760.63,C333676753,0.00,0.00,C1564353608,3198359.45,4750120.08,0,...,False,True,0,0.495751,60,High-risk transaction type; Large transaction ...,0.6,0.537450,0,Not investigated (low risk)
4,4,206,78172.30,C813403091,2921331.58,2999503.88,C1091768874,415821.90,337649.60,0,...,False,False,0,0.224300,0,NaN,0.0,0.134580,0,Not investigated (low risk)


In [22]:
bad_df=df[df['llm_explanation'] != "Not investigated (low risk)"]
bad_df.head()

,transaction_id,step,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,...,type_PAYMENT,type_TRANSFER,ml_anomaly_flag,ml_fraud_score,rule_score,rule_reasons,rule_score_norm,total_fraud_score,flagged_fraud,llm_explanation
331,331,684,4126119.93,C625476444,344527.00,0.0,C188307334,446963.89,4573083.81,0,...,False,True,1,0.781343,100,Sender balance drained to zero; High-risk tran...,1.0,0.868806,1,Risk Level: High\n\nKey Risk: The transaction ...
1091,1091,403,5279372.81,C1900381215,2527.46,0.0,C900921479,6197777.70,11477150.50,0,...,False,True,1,0.778405,100,Sender balance drained to zero; High-risk tran...,1.0,0.867043,1,Risk Level: High\n\nKey Risk: The high ML Frau...
1345,1345,538,3856775.88,C1611333212,52718.00,0.0,C952186896,1329128.71,5185904.59,0,...,False,True,1,0.690395,100,Sender balance drained to zero; High-risk tran...,1.0,0.814237,1,Risk Level: High\n\nKey Risk: The transaction ...
1632,1632,304,10000000.00,C1965389823,164.00,0.0,C746161132,566741.88,17627555.08,0,...,False,True,1,0.822525,100,Sender balance drained to zero; High-risk tran...,1.0,0.893515,1,Risk Level: High\n\nKey Risk: The transaction ...
2530,2530,375,10001912.39,C2137629092,411395.34,0.0,C1128848873,16677560.66,26679473.05,0,...,False,True,1,0.956039,100,Sender balance drained to zero; High-risk tran...,1.0,0.973624,1,Risk Level: High\n\nKey Risk: The transaction'...


In [23]:
output_path=r"C:\Users\MAITHILI\Gen-AI_FraudGuard\Datasets\paysim_processed_with_scores.csv"

df.to_csv(output_path, index=False)

print("Saved processed dataset to:", output_path)
print("Final shape:", df.shape)

Saved processed dataset to: C:\Users\MAITHILI\Gen-AI_FraudGuard\Datasets\paysim_processed_with_scores.csv
Final shape: (150000, 25)
